# Lesson 8: Machine Learning - Classification, Dimesionality Reduction

You have already started Machine Learning when you performed the linear regression analysis, but let's talk about Machine Learning in general first, then, as promised earlier, we'll move to our larger dataset. 

<table style="width:100%;border: 1px solid black;padding: 8px;">
	<tr>
        <th rowspan=4><p style="text-align:center;font-size:200%;"> Machine Learning</p><p style="text-align:center;">(Learn from data and make decisions)</p></th>
        <th rowspan=2><p style="text-align:center;font-size:200%;"> Supervised Learning </p><p style="text-align:center;">(Predictive Model)</p></th>
        <td style="text-align:center;"> Classification </td>
	</tr>
    <tr>
    	<td style="text-align:center;"> Regression </td>
    </tr>
    <tr>
        <th rowspan=2><p style="text-align:center;font-size:200%;"> Unsupervised Learning</p><p style="text-align:center;">(Non-predicitve Model)</p></th>
        <td style="text-align:center;"> Clustering </td>
	</tr>
    <tr>
        <td style="text-align:center;"> Dimensionality Reduction </td>
	</tr>
</table>


### Supervised Learning: 
Use training set with correct inputs and outputs to predict outputs for test data inputs. 
#### Classification: 
 - Inputs(X): Features 
 - Outputs(y): binary or multiple classes
 
#### Regression: 
 - Inputs(X): Independent Variable 
 - Outputs(y): Dependent Variable (Continuous)
 
### Unupervised Learning:
Find patterns among inputs (features), no labels in data
#### Clustering:
- Find groups within data (Example: Phylogeny tree)

#### Dimensionality Reduction:
- Find a lower dimension representation of higher dimensional data



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pwd

In [ ]:
# Remember : You are declaring the dataframes here, rerun if dataframes get messed up
testX = pd.read_csv("/kaggle/input/gene-expression/data_set_ALL_AML_independent.csv")
trainX = pd.read_csv("/kaggle/input/gene-expression/data_set_ALL_AML_train.csv")
labels = pd.read_csv("/kaggle/input/gene-expression/actual.csv") # contains Y (labels) for both train and test

# Explore the Data 
- Get the first 3 lines of training data using```trainX.head(n=3)``` (and test data similarly)
- Get the full list of columns in train and test
- Get a feel of the datasets

In [ ]:
trainX.head(n=3)


In [ ]:
testX.head(n=3)


For each of the three datasets, get the
-  Number of rows
- Numbeer of Columns
- Title of columns

In [ ]:
print(len(testX), "rows, and", len(testX.columns),"columns")
print(testX.columns)

In [ ]:
print(trainX.shape)
print(trainX.columns)

In [ ]:
print(len(labels), "rows, and", len(labels.columns),"columns")
print(labels.columns)

In [ ]:
#@title Data Imputation
# are there any NA values

any([trainX[col].isna().any() for col in trainX.columns]) # We're asking "Yo! Any NAs?"

# Luckily, nah!
# There's functions like dropna() that you can use to deal with NAs

# Data Cleaning and manipulation
### 1) We don't need those 'call.\*' columns
How do you remove all those particular columns with column name of 'call.xx' type?

### 2) What are the rows and what are the columns?
Better representation is probably with genes as columns and samples as rows (Transpose the data)

### 3) Can we separate the gene descriptions and gene accession numbers?
Both test and train share these two row (after transpose) and we could keep them separate from numerical contents of test and train.
### 4) Are the column names in order?
Sort the columns by column names

In [ ]:
genes = trainX.iloc[:,:2]

train_X = trainX[ sorted([valid_name for valid_name in trainX.columns[2:] if valid_name[:4] != 'call'], key = lambda colname: int(colname) ) ].T
test_X = testX[ sorted([valid_name for valid_name in testX.columns[2:] if valid_name[:4] != 'call'], key = lambda colname: int(colname) ) ].T

### 5) Make the labels 0 for ALL 1 for AML and get train_Y (first 38) test_Y (38 onwards)

In [ ]:
train_Y, test_Y =  1*(labels[:38]["cancer"] == "AML"), 1*(labels[38:]["cancer"] == "AML")

# Data visualization
Well, let's plot the train_X and show train_Y labels.
But wait how many features do you need to show? Now how do we show these **dimensions**?
Turns out we can only visualize 2 or 3 dimensions.
Solution: perform **Dimensionality Reduction**

### Dimensionality Reduction
We find a lower dimensional representation data that best explains the variation in the data.

<div align="center">
    <figure class="image"><img src="https://miro.medium.com/max/1024/1*vfLvJF8wHaQjDaWv6Mab2w.png" alt="kNN example" width="600">
        <figcaption> <h4> Figure 1: Dimensionality Reduction example: Principal Component Analysis transforming 3D to 2D (adapted from a medium article by SaiGayatri Vadali) </h4>
        </figcaption>
    </figure>
</div>


Some algorithms for Dimensionality Reduction include PCA, U-MAP, t-SNE

In [ ]:
from sklearn.decomposition import PCA
reducer3 = PCA(n_components = 3)
reducer2 = PCA(n_components = 2)
train_X_PCA3 = reducer3.fit_transform(train_X)
train_X_PCA2 = reducer2.fit_transform(train_X)

- Check the shape of both both reduced vectors
- Do you find any similarity in the columns of the two vectors


In [ ]:
train_X_PCA3, train_X_PCA2

Now let's make a 3-D plot, using the three principal comonents of train data as features, and show corresponding labels as colors

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

fig = plt.figure(figsize =(10,6))
ax = plt.axes(projection='3d')

ax.scatter3D(train_X_PCA3[:,0], train_X_PCA3[:,1], train_X_PCA3[:,2], c=np.array([train_Y]).T);

Similarly, can you make a 2-D plot of our training data and test data combined? Name the principal components vector (which will be of shape (72,2)) ```X_PCA2```

*Make sure you get the two labels forming largely visibly distinct clusters. Ask for help if you dont't*

*Hint: Should you PCA after or before combining train and test data*


<details><summary>Try yourself and then find the solution code here</summary>

X = np.vstack([train_X,test_X]) <br>
X_PCA2 = reducer2.fit_transform(X) <br>
Y = np.hstack([[train_Y],[test_Y]]).T <br>
plt.scatter(X_PCA2[:,0],X_PCA2[:,1],c=Y) <br>
    or<br>
train_X_PCA2 = reducer2.fit_transform(train_X) <br>
test_X_PCA2 = reducer2.transform(test_X) <br>
X_PCA2 = np.vstack([train_X_PCA2,test_X_PCA2]) <br>
Y = np.hstack([[train_Y],[test_Y]]).T <br>
plt.scatter(X_PCA2[:,0],X_PCA2[:,1],c=Y) <br>
</details>


In [ ]:
plt.scatter(train_X_PCA2[:,0], train_X_PCA2[:,1], c=np.array([train_Y]).T);


Now that you understand how dimensionality reduction basically works, you can even use UMAP and t-SNE without even reading about the underlying algorithms, almost like a black box. UMAP projections are very widely used nowadays to plot scRNA seq data for different cell populations and compare different cell types. Feel free to use scikit APIs for UMAP and t-SNE. 
UMAP 
Refer to [sckit-learn website](https://scikit-learn.org/stable/) to explore

# Classification
Classification is a supervised learning where you predict particular labels, here "AML" versus "ALL" (technically 0 or 1). While we are performing binary classification, i.e. comparing probability of getting 0 or 1, in multi-class classfication, you would compare probability of getting more than 2 labels, and predict out of different classes.

## k-Nearest Neighbors
One simple classification approach is to look at k nearest training data points to a test data point, and choose the label most common among the k-nearest neighbors. Of course, your results can eaaily vary with the choice of k parameter.


<div align="center">
    <figure class="image"><img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2018/03/knn3.png" alt="kNN example" width="600">
        <figcaption> <h4> Figure 2: k-Nearest Neighbors example. Green dot is the test data point. If k = 3, prediction is triangle, but if k = 5 prediction is square. (adapted from analyticsvidhya.com)</h4>
        </figcaption>
    </figure>
</div>


### Get the knn implememntation from sklearn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as knn

In [ ]:
# Initiate KNeighborsClassifier object
threeNN = knn(n_neighbors=3)

### Train (Fit) the classifier with X as train_x (all but first two rows) and train_y

In [ ]:
threeNN.fit(train_X, train_Y)

### Get prdicted labels for test_X ( This are predicted y values or $\hat{y}$)

In [ ]:
test_Y_hat = threeNN.predict(test_X)

### Awesome! Now let's see how well did our model really do?
Accuracy is the most straight-forward metric of model performance, a confusion matrix gives us:

<table>
<tr>
    <td style="color:green"> True Positive </td>
    <td style="color:red"> False Positive </td>
</tr>

<tr>
    <td style="color:red">False Negative</td>
    <td style="color:green">True Negative</td>
</tr>
</table>

In [ ]:
# Get the accuracy
print("Accuracy: ",np.sum((test_Y_hat - test_Y) == 0)/len(test_Y))
# Get the 'confusion matrix' using sklearn.metrics
from sklearn.metrics import confusion_matrix, accuracy_score,plot_confusion_matrix
confusion_matrix(test_Y, test_Y_hat)

In [ ]:
plot_confusion_matrix(threeNN,test_X,test_Y)

In [ ]:
def kNN(k, train_X, train_Y,test_X,test_Y):
    return  accuracy_score(knn(n_neighbors=k).fit(train_X, train_Y).predict(test_X),test_Y)

In [ ]:
kNN(3, train_X, train_Y,test_X,test_Y)


So now we built a basic classifier. There is significant scope to improve, or **_preprocess_** our model.
Some approaches you should try to use:
### 1) Standardize the data: 
Is it fair for genes with higher average expression to weigh as much as genes with lower average expression for the same change in expression? If we have some genes that show high fluctuations in expression irrespective of AML or ALL, could their changes 'confuse' our model, or make it difficult to training our model. When your features might not be in the same range, it might be helpful to center the data by the mean of training data and scale it by standard deviation of training  data (which is just the z-scores for training data, and hopefully close to z-scores for test data). This can be done by sklearn.preprocessing.StandardScaler However, if your data has many outliers, you should scale 



In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled = scaler.transform(test_X)

from sklearn.preprocessing import RobustScaler
rscaler = RobustScaler()
train_X_rscaled= rscaler.fit_transform(train_X)
test_X_rscaled = rscaler.transform(test_X)

### 2) Reduce Dimensions
With 7129 features for each of the 38 training data points, how many numbers do you think the k-NN algorithm is using right now? This is called the curse of dimensionality. Now that we know how to project our data to lower dimensions using PCA, do you think we could use the two principal components as input features for each training data point? While this is not always the case, we know this should work for our data because when we visualized the data (2D plot), the AML- and ALL-labelled points were prety well separated. However, remember that PCA is unsupervised learning it doesn't take into account the labels and could have thus done rather more harm than good by making the classes overlap instead of separate. [Click here](https://cdn.analyticsvidhya.com/wp-content/uploads/2017/03/06064252/Image_cont_26.jpg) to see an example of that circumstance.

In [ ]:
train_X_PCA2 = reducer2.fit_transform(train_X_rscaled)
test_X_PCA2 = reducer2.transform(test_X_rscaled)

In [ ]:
# k = 1,3,5,15, 25
# standard scaler
# pca
results = pd.DataFrame(np.zeros([5,4]),columns = ["basic","standard-scaled","robust-scaled","PCA reduced"], index=[1,3,5,15,25])
for k in [1,3,5,15,25]:
    results.loc[k] = [kNN(k, train_X, train_Y,test_X,test_Y),kNN(k, train_X_scaled, train_Y,test_X_scaled,test_Y),kNN(k, train_X_rscaled, train_Y,test_X_rscaled,test_Y),kNN(k, train_X_PCA2, train_Y,test_X_PCA2,test_Y)]

In [ ]:
results

In [ ]:
for col in results.columns:
    plt.plot(results.index,results[col], label = col)
plt.legend()

### Which preprocessing stepa and which ```k``` parameter gives you the best classifier?
### How does the accuracy vary with k?


What if we wanted to know which genes are more contributing in differentiating AML from ALL?

Then another classification algorithm called Logistic Regression might be useful. (*Note that this regression is actually used for classification*)

Logistic Regression tries to fit a S-shaped ([Sigmoid curve](https://en.wikipedia.org/wiki/Sigmoid_function) ) over the data. The sigmoid curve has asymptotes at y = 0 and y = 1, and the best-fitting simoid curve is obtained by optimizing the weights associated with each feature (each gene here). ![](https://sebastianraschka.com/images/faq/logistic-why-sigmoid/5.png) ![](https://sebastianraschka.com/images/faq/logistic-why-sigmoid/7.png)
#### Fig 3: Sigmoid curve equation and graph
Then we can interpret the weights (the coefficients) as the predicitive power a feature.
### Task: Get the top 10 determining genes in our example

In [ ]:
from sklearn.linear_model import LogisticRegression
LRclf = LogisticRegression()
LRclf.fit(train_X, train_Y)
# LRclf.fit(train_X_scaled, train_Y)

In [ ]:
# to keep track of indices of genes, we use enumerate()
print(accuracy_score( LRclf.predict(test_X),test_Y))
sorted(enumerate(np.squeeze(LRclf.coef_)), key = lambda x: x[1])[:7129 -16:-1]

In [ ]:
top15AMLindices = np.array(sorted(enumerate(np.squeeze(LRclf.coef_)), key = lambda x: x[1])[:7129 -16:-1])[:,0]
genes.iloc[top15AMLindices]

In [ ]:
top15ALLindices = np.array(sorted(enumerate(np.squeeze(LRclf.coef_)), key = lambda x: x[1])[:15])[:,0]
genes.iloc[top15ALLindices]

To confirm this visually, let us make a heatmap of expression level of these top 15 ALL-contributing and top 15 AML-contributing genes for 10 AML patients, and 10 ALL patients.
### Interpeting the heatmap of selected genes
Which regions do you find warmer for the first 27 columns (ALL samples)? The top 15 genes or the bottom 15?
Which regions do you find warmer for the last 11 columns (AML samples)? The top 15 genes or the bottom 15?

In [ ]:
import seaborn as sns
import scipy.stats as ss

indices = np.hstack([top15ALLindices,top15AMLindices]) # so we first take 15 ALL-contributing genes, then 15 AML-contributing genes

train_X_std = pd.DataFrame(ss.zscore(train_X.iloc[:,indices]), columns = [gene_desc[:30] for gene_desc in genes.iloc[indices,0]]) # Remember our training data has top 27 rows ALL, then rest AML

plt.figure(figsize = (10,6))
sns.heatmap(train_X_std.T,linewidths=.05,cmap="coolwarm")
plt.xlabel("Traning sampeles")